# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
df = pd.read_csv('homepage_actions.csv')

In [2]:
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
df['action'].value_counts()

action
view     6328
click    1860
Name: count, dtype: int64

In [4]:
clicked = set(df[df['action'] == 'click']['id'].unique())
viewed = set(df[df['action'] == 'view']['id'].unique())
print('number of clicks : {}'.format(len(clicked)))
print('number of viewers : {}'.format(len(viewed)))
print('number of viewes who didnt click : {}'.format(len(viewed - clicked)))
print('number of viewers that did click : {}'.format(len(viewed - (viewed - clicked))))

number of clicks : 1860
number of viewers : 6328
number of viewes who didnt click : 4468
number of viewers that did click : 1860


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [5]:
df['count'] = 1
control = df[df['group'] == 'control'].pivot(index = 'id', columns = 'action', values = 'count')
control = control.fillna(value = 0)
experiment = df[df['group'] == 'experiment'].pivot(index = 'id', columns = 'action', values = 'count')
experiment= experiment.fillna(value = 0)

print('sample sizes : \n control : {}\n experimental : {}'.format(len(control), len(experiment)))
print('')
print('total clicks : \n control : {}\n experimental : {}'.format(control.click.sum(), experiment.click.sum()))
print('')
print('average click rate : \n control : {}\n experimental : {}'.format(control.click.mean(), experiment.click.mean()))

sample sizes : 
 control : 3332
 experimental : 2996

total clicks : 
 control : 932.0
 experimental : 928.0

average click rate : 
 control : 0.2797118847539016
 experimental : 0.3097463284379172


In [19]:
def one_sample_var(sample):
    sample_mean = sample.mean()
    return np.sum((sample.mean()**2) / (len(sample) - 1))
def pooled_var(sample1, sample2):
    l_1, l_2 = len(sample1), len(sample2)
    var_1, var_2 = one_sample_var(sample1), one_sample_var(sample2)
    return ((l_1 - 1) * var_1 + (l_2 - 1) * var_2) / ((l_1 + l_2) - 2)
def two_samp_t_stat(sample1,sample2):
    c_mean, e_mean = sample1.mean(), sample2.mean()
    p_var = pooled_var(sample1,sample2)
    l_c, l_e = len(sample1), len(sample2)
    num = e_mean - c_mean
    denom = np.sqrt(p_var * ((1 / l_e) + (1 / l_c)))
    return num / denom

In [20]:
print(one_sample_var(control.click))
print(pooled_var(control.click, experiment.click))
print(two_samp_t_stat(control.click, experiment.click))

2.3488063186004178e-05
2.753422801981505e-05
227.33866503040892


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [21]:
control_rate = control.click.mean()
e_c_rate = control_rate * len(experiment)
e_c_rate

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [25]:
n = len(experiment)
p = control_rate
var = n * p * (1 - p)
std = np.sqrt(var)
std

24.568547907005815

In [26]:
e_c = experiment.click.sum()
z = (e_c - e_c_rate) / std
z

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [27]:
p_val = stats.norm.sf(z)
p_val

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.